In [29]:
from glob import glob
import numpy as np

In [30]:
from rdkit import Chem

In [31]:
target_xyzs = sorted(glob("targets/*.xyz"))

In [32]:
def read_sdf(sdf):
    with open(sdf, "r") as f:
        txt = f.read().rstrip()
    return txt

In [33]:
def get_ncharges_coords(sdf):
    mol = Chem.MolFromMolBlock(sdf)
   #mol = Chem.AddHs(mol)
    # rdkit molobj
    ncharges = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    conf = mol.GetConformer()
    coords = np.asarray(conf.GetPositions())
    return ncharges, coords

In [34]:
def cutoff_func(R_ij, central_cutoff=4.8, central_decay=1):
    if R_ij <= (central_cutoff - central_decay):
        func = 1.
    elif ((central_cutoff - central_decay) < R_ij) and (R_ij <= (central_cutoff + central_decay)):
        func = 0.5 * (1. + np.cos((np.pi * R_ij - central_cutoff + central_decay)/central_decay))
    else:
        func = 0.
    return func

In [35]:
def get_atomic_CM(ncharges, coords, max_natoms, central_cutoff=4.8, central_decay=1):
    size = int((max_natoms + 1)*max_natoms / 2)
    rep = np.zeros((len(ncharges), size))
    
    # central atom loop
    for k in range(len(ncharges)):
        M = np.zeros((len(ncharges), len(ncharges)))
        for i in range(len(ncharges)):
            R_ik = np.linalg.norm(coords[i]-coords[k])
           # print('R_ik', R_ik)
            f_ik = cutoff_func(R_ik, central_cutoff=central_cutoff,
                              central_decay=central_decay)
            for j in range(len(ncharges)):
                if i <=j:
                    if i == j:
                        M[i,j] = 0.5 * ncharges[i]**2.4 * f_ik**2
                        M[j,i] = M[i,j]

                    else:
                        R_jk = np.linalg.norm(coords[j]-coords[k])
                      #  print('R_jk', R_jk)
                        f_jk = cutoff_func(R_jk, central_cutoff=central_cutoff,
                                          central_decay=central_decay)
                        R_ij = np.linalg.norm(coords[i]-coords[j])
                      #  print('R_ij', R_ij)
                        f_ij = cutoff_func(R_ij, central_cutoff=central_cutoff,
                                          central_decay=central_decay)
                        M[i,j] = (ncharges[i]*ncharges[j]/R_ij)*f_ik*f_jk*f_ij
                        M[j,i] = M[i,j]


        # concat upper triangular and diagonal
        upper_triang = M[np.triu_indices(len(M))]
        s_upper_triang = np.sort(upper_triang)[::-1]
        
        # pad to full size
        n_zeros = size - len(s_upper_triang)
        zeros = np.zeros(n_zeros)
        rep[k] = np.concatenate((s_upper_triang, zeros))

    return rep

In [36]:
target_files = sorted(glob("targets/*.sdf"))
target_files

['targets/qm9.sdf', 'targets/vitc.sdf', 'targets/vitd.sdf']

In [37]:
target_sdfs = [read_sdf(x) for x in target_files]

In [38]:
conf_data = [get_ncharges_coords(x) for x in target_sdfs]

In [39]:
ncharges_list, coords_list = zip(*conf_data)

In [40]:
ncharges_list[0]

[8, 6, 6, 7, 6, 8, 8, 7, 6]

In [41]:
sizes = [len(x) for x in ncharges_list]
sizes

[9, 12, 28]

In [42]:
target_reps = np.array(
[np.array(get_atomic_CM(np.array(ncharges_list[i]), np.array(coords_list[i]),
                                max_natoms=sizes[-1]))
for i in range(len(ncharges_list))])

/home/puck/anaconda3/envs/rdkit/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


In [43]:
target_reps[0]

array([[73.51669472, 53.3587074 , 43.58776225, ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 53.3587074 , 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 73.51669472, 73.51669472, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [73.51669472, 73.51669472, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ]])

In [44]:
target_labels = [t.split("/")[-1].split(".xyz")[0] for t in target_sdfs]

In [45]:
np.savez("target_aCM_data.npz", 
         target_labels=target_labels, 
         target_reps=target_reps, 
         target_ncharges=ncharges_list,)